Первый вариант решения с помощью n-грамной модели

In [1]:
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import re
from collections import Counter

Для создания и обучения модели я буду испоьзовать несколько текстов книг: Гарри Поттера и Хоббита

In [2]:
with open ('64735.txt', encoding="utf-8") as text_file:
    text = text_file.readlines()
text = ' '.join(text)
text = text.lower()

С помощью регулярных выражений произвожу отбор всех слов

In [3]:
import re
pattern = r'\b[а-яё]+\b'
words = re.findall(pattern, text)

Первым делом создадим словарь, в котором каждому слову будет присвоен определенный уникальный индекс. А также создадим обратный словарь, где каждому индексу будет соответствовать свое слово. А также создадим последовательность слдов в исходном тексте

In [5]:
cnt = Counter(words)
counts = sorted(cnt.items(), key = lambda x: x[1], reverse=True)
only_words_sorted = [word for word, summ in counts]
word_to_ind = {word: index for index, word in enumerate(only_words_sorted)}
ind_to_word = {value: key for key, value in word_to_ind.items()}
seq = np.array([word_to_ind[word] for word in words])

Создадим N-граммную модель

In [7]:
d = {}
for i in range(len(words)-1):
    key = words[i]
    if key not in d.keys():
        d[key] = []
        d[key].append(words[i+1])
    else:
        d[key].append(words[i+1])

In [8]:
n_mod = {}
for key, value in d.items():
    new_val = []
    for word in value:
        p = value.count(word)/len(value)
        word_p = (word, p)
        new_val.append(word_p)
    
    n_mod[key] = list(set(new_val))

In [9]:
n_mod_2 = {}
for key, value in n_mod.items():
    key = word_to_ind[key]
    arr_words = []
    arr_p = []
    for pair in value:
        arr_words.append(word_to_ind[pair[0]])
        arr_p.append(pair[1])
    n_mod_2[key] = [arr_words, arr_p]

В работе я буду использовать n_mod_2, где во-первых слова зашифрованы в индексы, а во вторых этот словарь содержит в себе два списка: следующие за словом слова и соответсятвующие им вероятности

Затем я создаю функцию, для продолжения текста. Она принимает на вход начало текста и количество слов ожидаемого текста. Возвращает соответсятвенно текст с продолжением

In [12]:
def text_continue(beggining, l):
    l = l-1
    k_w = word_to_ind[beggining[-1]]
    if sum(n_mod_2[k_w][1]) != 1:
        n_mod_2[k_w][1][0] += 1 - sum(n_mod_2[k_w][1])
    top_ind = np.random.choice(n_mod_2[k_w][0], p=n_mod_2[k_w][1])
    beggining.append(ind_to_word[top_ind])
    if l > 0:
        text_continue(beggining, l)
    return ' '.join(beggining)

Затем непосредственно генерация текста

In [16]:
beg = input().lower().split()
size = int(input())
print(text_continue(beg, size))

мой хороший
10
мой хороший парень гарри эти слухи о тролле старой распре между ними


Да, текст полученный на выходе выглядит глупо и несуразно, так как не проводилось никакого обучения и "работы над ошибками", соответственно здесь мы имеем дело практически с рандомными последовательностями